# imports

In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import warnings

from datetime import date
from itertools import product
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

warnings.simplefilter('ignore')

# Config

In [3]:
DATA_FILE_PATHS = '/content/drive/MyDrive/RC/data/'
START_DATE = '2023-01-01'
TEST_DATE = '2023,4,1'
LAST_DATE = '2023,5,1'
FEATURE_LIST = [
    'PULocationID',
    'PU_day_of_month',
    'PU_day_of_week',
    'last_day_demand',
    'last_week_demand'
]
TARGET = 'count'
VALIDATION_SPLIT_RATIO = 0.2
LR_OUTPUT_PATH = '/content/drive/MyDrive/RC/output/lr_result.parquet'
XGB_OUTPUT_PATH = '/content/drive/MyDrive/RC/output/XGB_result.parquet'

# Load Data

In [4]:
def load_data(file_paths, start_date = None):
    df = pd.read_parquet(file_paths)
    df['date'] = df['tpep_pickup_datetime'].dt.date.astype(str)

    if start_date:
        df = df[df['date'] > start_date].reset_index(drop = True)
    return df

rides_df = load_data(
    DATA_FILE_PATHS,
    START_DATE
)

print(rides_df.shape)
rides_df.head()

(12595923, 20)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,date
0,2,2023-01-02 00:00:37,2023-01-02 00:13:15,1.0,9.29,1.0,N,70,4,1,35.9,1.0,0.5,8.18,0.00,1.0,49.08,2.5,0.00,2023-01-02
1,2,2023-01-02 00:01:53,2023-01-02 00:34:16,1.0,20.40,2.0,N,132,238,1,70.0,0.0,0.5,15.86,6.55,1.0,95.16,0.0,1.25,2023-01-02
2,2,2023-01-02 00:04:59,2023-01-02 00:12:03,5.0,1.68,1.0,N,142,229,1,10.0,1.0,0.5,2.25,0.00,1.0,17.25,2.5,0.00,2023-01-02
3,2,2023-01-02 00:00:28,2023-01-02 00:08:45,1.0,1.74,1.0,N,164,224,1,10.7,1.0,0.5,0.00,0.00,1.0,15.70,2.5,0.00,2023-01-02
4,2,2023-01-02 00:00:08,2023-01-02 00:04:30,6.0,0.63,1.0,N,144,231,1,6.5,1.0,0.5,0.00,0.00,1.0,11.50,2.5,0.00,2023-01-02


# aggregate data and labeling

In [5]:
def labeling(rides_df : pd.DataFrame):
    aggregated_df = rides_df.groupby(
        ['date', 'PULocationID']).size().reset_index(name='count')
    unique_dates = rides_df['date'].unique()
    unique_pu_location_ids = rides_df['PULocationID'].unique()
    all_combinations = list(
        product(
            unique_dates,
            unique_pu_location_ids
        )
    )
    combinations_df = pd.DataFrame(
        all_combinations,
        columns=['date', 'PULocationID']
    )
    label_df = aggregated_df.merge(
        combinations_df,
        how='right',
        on=['date', 'PULocationID']
    ).fillna(0)

    return label_df

rides_df = labeling(rides_df)

print(rides_df.shape)
rides_df.head()

(31964, 3)


,date,PULocationID,count
0,2023-01-02,70,503.0
1,2023-01-02,132,6419.0
2,2023-01-02,142,2028.0
3,2023-01-02,164,1462.0
4,2023-01-02,144,567.0


# Feature Extraction

## adding calender features

In [6]:
def adding_feature(rides_df : pd.DataFrame):
    rides_df['date'] = rides_df['date'].astype('datetime64')
    rides_df['PU_day_of_month'] = rides_df['date'].dt.day.astype(np.uint8)
    rides_df['PU_day_of_week'] = rides_df['date'].dt.weekday.astype(np.uint8)
    rides_df = rides_df.sort_values(['date'])
    rides_df['last_day_demand'] = rides_df.groupby(['PULocationID'])['count'].shift(1)
    rides_df['last_week_demand'] = rides_df.groupby(['PULocationID'])['count'].shift(7)
    return rides_df

rides_df['count'] = rides_df['count'] + 1
rides_df = adding_feature(rides_df)

print(rides_df.shape)
rides_df.head()

(31964, 7)


,date,PULocationID,count,PU_day_of_month,PU_day_of_week,last_day_demand,last_week_demand
0,2023-01-02,70,504.0,2,0,NaN,NaN
166,2023-01-02,165,2.0,2,0,NaN,NaN
167,2023-01-02,3,3.0,2,0,NaN,NaN
168,2023-01-02,147,3.0,2,0,NaN,NaN
169,2023-01-02,122,3.0,2,0,NaN,NaN


## checking one week of data as a sample

In [7]:
rides_df[(rides_df['PULocationID'] == 79)].tail(8)

,date,PULocationID,count,PU_day_of_month,PU_day_of_week,last_day_demand,last_week_demand
29108,2023-04-26,79,2013.0,26,2,1678.0,1852.0
30418,2023-04-27,79,2261.0,27,3,2013.0,2293.0
30942,2023-04-28,79,3146.0,28,4,2261.0,2977.0
31204,2023-04-29,79,4582.0,29,5,3146.0,4642.0
31466,2023-04-30,79,3350.0,30,6,4582.0,3103.0
31728,2023-05-01,79,1.0,1,0,3350.0,1422.0
30156,2023-05-02,79,2.0,2,1,1.0,1678.0
30680,2023-05-03,79,2.0,3,2,2.0,2013.0


## Dropping some samples

In [8]:
rides_df = rides_df.dropna()
date = LAST_DATE.split(',')
end_date_time = datetime.datetime(
    int(date[0]),
    int(date[1]),
    int(date[2])
)
rides_df = rides_df[rides_df['date'] < end_date_time]

print(rides_df.shape)
rides_df.head()

(29344, 7)


,date,PULocationID,count,PU_day_of_month,PU_day_of_week,last_day_demand,last_week_demand
2008,2023-01-09,255,5.0,9,0,52.0,18.0
2011,2023-01-09,181,17.0,9,0,9.0,32.0
2010,2023-01-09,39,12.0,9,0,11.0,4.0
2009,2023-01-09,117,6.0,9,0,3.0,5.0
2007,2023-01-09,85,4.0,9,0,4.0,5.0


## Train and Test split

In [9]:
def train_and_test_split(data, split_date):

  date = split_date.split(',')
  start_date_time = datetime.datetime(
      int(date[0]),
      int(date[1]),
      int(date[2])
  )
  train_data = data[
      rides_df['date'] < start_date_time
  ]
  test_data = data[
      rides_df['date'] >= start_date_time
  ]

  train_data.set_index('date', inplace = True)
  test_data.set_index('date', inplace = True)

  return train_data, test_data

train_df, test_df = train_and_test_split(
    rides_df,
    TEST_DATE
)

print(train_df.shape)
print(test_df.shape)
train_df.head()

(21484, 6)
(7860, 6)


,PULocationID,count,PU_day_of_month,PU_day_of_week,last_day_demand,last_week_demand
date,,,,,,
2023-01-09,255,5.0,9,0,52.0,18.0
2023-01-09,181,17.0,9,0,9.0,32.0
2023-01-09,39,12.0,9,0,11.0,4.0
2023-01-09,117,6.0,9,0,3.0,5.0
2023-01-09,85,4.0,9,0,4.0,5.0


## Target and Feature split

In [10]:
train_label_df = train_df[TARGET]
train_df = train_df[FEATURE_LIST]

test_label_df = test_df[TARGET]
test_df = test_df[FEATURE_LIST]

## Train and Validation split

In [11]:
train_df, validation_df, train_label_df, validation_label_df = train_test_split(
    train_df,
    train_label_df,
    test_size = VALIDATION_SPLIT_RATIO,
    shuffle = False
)

# ML Models

In [12]:
def model_training(ml_model, train_df, train_label_df, **params):
  model = ml_model(**params)
  model.fit(
      train_df,
      train_label_df
  )
  return model

replace_negatives = np.vectorize(lambda x : 0 if x < 0 else x)

## Calculate Error

In [13]:
def symmetric_mean_absolute_percentage_error(actual, predicted) -> float:
	return round(
      np.mean(
          np.abs(predicted - actual) /
          ((np.abs(predicted) + np.abs(actual)) / 2)
      ), 4
  )

def error_calculator(real_demand, predicted_demand):
  print(
      'SMAPE: ',
      round(
          symmetric_mean_absolute_percentage_error(
              real_demand,
              predicted_demand
          ) * 100 , 2
      ), '%'
  )
  print(
      'MAPE:  ',
      round(
          float(
              mean_absolute_percentage_error(
                  real_demand,
                  predicted_demand
              )
          ) * 100, 2
      ), '%'
  )
  print(
      'MSE:   ',
      round(
          float(
              mean_squared_error(
                  real_demand,
                  predicted_demand
              )
          ), 2
      )
  )
  print(
      'MAE:   ',
      round(
          float(
              mean_absolute_error(
                  real_demand,
                  predicted_demand
              )
          ), 2
      )
  )

## Linear Regression Model

In [14]:
lr_model = model_training(
    LinearRegression,
    train_df,
    train_label_df
)

### Validation prediction

In [15]:
lr_validation_pred = replace_negatives(
    np.round_(
        lr_model.predict(
            validation_df
        )
    )
)
error_calculator(
    validation_label_df,
    lr_validation_pred
)

SMAPE:  87.18 %
MAPE:   95.64 %
MSE:    13522.41 %
MAE:    37.0 %


### Test prediction

In [16]:
lr_test_pred = replace_negatives(
    np.round_(
        lr_model.predict(
            test_df
        )
    )
)
error_calculator(
    test_label_df,
    lr_test_pred
)

SMAPE:  86.24 %
MAPE:   144.77 %
MSE:    12663.37 %
MAE:    39.73 %


### Result Data

In [17]:
lr_result_df = test_df.copy()
lr_result_df['real demand'] = test_label_df
lr_result_df['predicted demand'] = lr_test_pred

print(lr_result_df.shape)
lr_result_df.head()

(7860, 7)


,PULocationID,PU_day_of_month,PU_day_of_week,last_day_demand,last_week_demand,real demand,predicted demand
date,,,,,,,
2023-04-01,27,1,5,1.0,1.0,1.0,-0.0
2023-04-01,132,1,5,6059.0,5191.0,5678.0,5429.0
2023-04-01,165,1,5,3.0,3.0,2.0,4.0
2023-04-01,3,1,5,6.0,1.0,1.0,1.0
2023-04-01,147,1,5,2.0,1.0,4.0,2.0


In [18]:
lr_result_df.to_parquet(LR_OUTPUT_PATH)

## XGBoost Model

### Hyperparameter tuning

In [19]:
def hyper_parameter_tuning(n_estimators, learning_rate, max_depth, scoring_method):
  parameters = {
      'n_estimators' : n_estimators,
      'learning_rate' : learning_rate,
      'max_depth' : max_depth
  }

  gc = GridSearchCV(
      XGBRegressor(),
      parameters,
      scoring = scoring_method
  )

  gc.fit(
      train_df,
      train_label_df
  )

  param = gc.best_params_

  return param

n_estimators = [700, 1000]
learning_rate = [0.15, 0.1, 0.01]
max_depth = [1, 2, 3]
scoring_method = 'neg_root_mean_squared_error'

param = hyper_parameter_tuning(
    n_estimators,
    learning_rate,
    max_depth,
    scoring_method
)

print(param)

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000}


### XGBoost Model

In [20]:
XGB_model = model_training(
    XGBRegressor,
    train_df,
    train_label_df,
    n_estimators = param['n_estimators'],
    learning_rate = param['learning_rate'],
    max_depth = param['max_depth']
)

### Validation prediction

In [21]:
XGB_validation_pred = replace_negatives(
    np.round_(
        XGB_model.predict(
            validation_df
        )
    )
)
error_calculator(
    validation_label_df,
    XGB_validation_pred
)

SMAPE:  45.89 %
MAPE:   68.12 %
MSE:    13021.16 %
MAE:    35.72 %


### Test prediction

In [22]:
XGB_test_pred = replace_negatives(
    np.round_(
        XGB_model.predict(
            test_df
        )
    )
)
error_calculator(
    test_label_df,
    XGB_test_pred
)

SMAPE:  45.47 %
MAPE:   79.76 %
MSE:    13297.87 %
MAE:    37.78 %


### Result Data

In [23]:
XGB_result_df = test_df.copy()
XGB_result_df['real demand'] = test_label_df
XGB_result_df['predicted demand'] = XGB_test_pred

print(XGB_result_df.shape)
XGB_result_df.head()

(7860, 7)


,PULocationID,PU_day_of_month,PU_day_of_week,last_day_demand,last_week_demand,real demand,predicted demand
date,,,,,,,
2023-04-01,27,1,5,1.0,1.0,1.0,3.0
2023-04-01,132,1,5,6059.0,5191.0,5678.0,5374.0
2023-04-01,165,1,5,3.0,3.0,2.0,3.0
2023-04-01,3,1,5,6.0,1.0,1.0,3.0
2023-04-01,147,1,5,2.0,1.0,4.0,3.0


In [24]:
XGB_result_df.to_parquet(XGB_OUTPUT_PATH)